# A Didactic STARK

This is a Python implementation of the STARK by Hand Explainer.
https://dev.risczero.com/proof-system/stark-by-hand

In [191]:
import random

import galois
import numpy as np
from sympy import intt

# We fix the random seed for reproducibility
random.seed(1)

## Security Warning

The finite field arithmetic implemented in galois are not constant-time, but were instead designed for performance. As such, the library could be vulnerable to a side-channel timing attack. This library is not intended for production security, but instead for research & development, reverse engineering, cryptanalysis, experimentation, and general education.

## Preliminaries

We use galois to define `_finite_field_array`. This functions like a `numpy` array, except that the array consists of elements of the finite field of size `_field_size`.

In [192]:
_field_size = 97
_finite_field_array = galois.GF(_field_size)
_primitive_root = _finite_field_array.primitive_element
_trace_length = 8
_reed_solomon_expansion_factor = 4
_block_length = _trace_length * _reed_solomon_expansion_factor
_generator = _primitive_root ** ((_field_size - 1) // _block_length)


# An EvaluationDomain is a collection of points on which to evaluate a polynomial
class EvaluationDomain:
    # Construct by directly providing the domain as an array
    def __init__(self, array):
        self.commitment = None  # Produced by evaluating a polynomial over self
        self.size = len(array)
        self.domain = array
        if self.size > 1:
            self.evaluation_domain_of_half_size = EvaluationDomain([self.domain[2 * i] for i in range(self.size // 2)])

    # Alternate constructor
    @classmethod
    def from_shift_generator_size(self, shift, generator, size):
        return EvaluationDomain(np.array([(shift * mod_pow(generator, i)) % _field_size for i in range(size)]))

    # Method to construct evaluation matrix for the given domain
    def as_vandermonde_matrix(self):
        self.evaluationmatrix = np.array(
            [[mod_pow(self.domain[j], i) for i in range(self.size)] for j in range(self.size)]
        )
        return self.evaluationmatrix

    # Method to evaluate a given polynomial in a coefficient form over the given domain
    def evaluate(self, coeffs):
        self.commitment = self.as_vandermonde_matrix() @ extendarray(coeffs, self.size) % _field_size
        return self.commitment


# A function that performs modular exponentiation


def mod_pow(base, exp):
    return pow(int(base), exp, _field_size)


# A function that returns the reciprocal modulo _field_size
def field_reciprocal(x):
    # By Fermat's Little Theorem, x^95 is the reciprocal of x (for non-zero x).
    x = _finite_field_array(x % _field_size)
    xinv = _finite_field_array(1) / x
    return xinv


# A function that returns an element-wise reciprocal of an array, modulo _field_size
def reciprocal_elementwise(array):
    return [field_reciprocal(x) for x in array]


# A function for extending a short array to a specified length:
def extendarray(short_array, length):
    size = len(short_array)
    # Validating short array is not too big:
    assert size <= length
    long_array = [0] * length
    # Write short_array contents into beginning of long_array:
    long_array[:size] = short_array[:size]
    return long_array


# A function that evaluates a polynomial at a given input
def evaluate_at_point(coeffs, x):
    # We evaluate f(x) as the dot product of [coefficients] and [powers]=[1, x, x^2, ...]
    powers = np.array([mod_pow(x, i) for i in range(len(coeffs))])
    eval = np.dot(coeffs, powers)
    return eval


# Define mix function for inside FRI
def mix(array, parameter):
    mixed_array = np.array([0] * len(array[0]))
    for i in range(len(array)):
        assert len(array[i]) == len(array[0])
        mixed_array = (mixed_array + (np.array(array[i]) * mod_pow(parameter, i))) % _field_size
    return mixed_array


# Define quotient function for DEEP
def DEEPquotient(block, domain, testpoint, eval_testpoint):
    eval_quotient = (block - eval_testpoint) * reciprocal_elementwise(domain - testpoint) % _field_size
    return eval_quotient


# Define a function that splits a block into 2 blocks of half the length
def split(block, large_domain):
    small_domain = large_domain.evaluation_domain_of_half_size
    # Re-write block as a galois array
    block = _finite_field_array(block)
    evalmatrix_inv = np.linalg.inv(_finite_field_array(large_domain.as_vandermonde_matrix()))

    # Given a block u, we construct smaller blocks v_0, v_1

    # The defining relationship between u and v_i is easy to state in coefficient representation:
    #   Writing u_bar as the coefficient form of u, and v_bar_i as the coefficient form of v_i, we write:
    #   u_bar(x) = v_bar_0(x^2) + (x * v_bar_1(x^2))
    # We compute the coefficients of v_bar_0 and v_bar_1 through the following matrix multiplication:
    # _finite_field_array is the finite field of order _field_size:

    # Computing coefficient form of input:
    u_bar = evalmatrix_inv @ block

    # Writing coefficients of output polynomials:
    v_bar_0 = extendarray([u_bar[2 * i] for i in range(len(u_bar) // 2)], len(small_domain.domain))
    v_bar_1 = extendarray([u_bar[1 + 2 * i] for i in range(len(u_bar) // 2)], len(small_domain.domain))

    # Computing output blocks
    v_0 = small_domain.evaluate(v_bar_0)
    v_1 = small_domain.evaluate(v_bar_1)

    # TODO(https://github.com/risc0/risc0/issues/979): Test and enable once verifier evaluation works
    # Instead of performing the matrix inversion and multiplication,
    # we directly split the evaluations into two blocks (even and odd indices)
    # v_0 = block[::2]  # even indices
    # v_1 = block[1::2]  # odd indices
    # v_0_evaluated = small_domain.evaluate(v_0)
    # v_1_evaluated = small_domain.evaluate(v_1)
    # return [v_0_evaluated, v_1_evaluated]

    return [v_0, v_1]


def fri(block, input_domain, mixing_parameters, total_rounds, rounds_remaining):
    while rounds_remaining > 0:
        print("\n--- begin round", total_rounds - rounds_remaining + 1, "---")
        print("D_super_(", total_rounds - rounds_remaining, "):", input_domain.domain)
        split0, split1 = split(block, input_domain)
        next_domain = input_domain.evaluation_domain_of_half_size
        print("Split blocks: \n", split0, "\n", split1)
        print("Now mix those blocks using the mix parameter", mixing_parameters[0])
        round_commitment = mix([split0, split1], mixing_parameters[0])
        print(
            "FRI Commitment (aka f_super_(",
            total_rounds - rounds_remaining + 1,
            "):",
            round_commitment,
        )
        return fri(
            round_commitment,
            next_domain,
            mixing_parameters[1:],
            total_rounds,
            rounds_remaining - 1,
        )
    if rounds_remaining == 0:
        print("D_super_(", total_rounds - rounds_remaining, "):", input_domain.domain)
        return block

# Lesson 1-3 : the Padded Execution Trace

In these lessons, we are proving computational integrity of a Fibonacci program mod `_field_size`.

We claim that the following trace is a valid instantiation of this algorithm. The trace is organized into 6 columns, which are split into two types: **Data** & **Control**.

For each row (in every column) we also append a random padding to the end of the execution trace, which adds the entropy to hide the user data.

## Trace: Data Columns

In this trace, we place intermediate computation steps for the Fibonacci program execution.

- We input `24` and `30` into a simple Fibonacci program. The inputs are loaded into `d1_trace[0]` and `d2_trace[0]`.
- We compute 4 rounds of Fibonacci addition. At each round, we compute:  `d3_trace[i] = d1_trace[i] + d2_trace[i]`
- After 4 rounds, `d3_trace[3]` contains the output of the computation.

In [193]:
rounds = 4

d1 = [24]
d2 = [30]
d3 = []
for i in range(rounds):
    d3.append((d1[i] + d2[i]) % _field_size)
    print(f"Round {i}: {d3[i]} = ({d1[i]} + {d2[i]}) % {_field_size}")
    if i < rounds - 1:
        # We don't need to append these traces in the last round
        d1.append(d2[i])
        d2.append(d3[i])

print("d1:", d1)
print("d2:", d2)
print("d3:", d3)

Round 0: 54 = (24 + 30) % 97
Round 1: 84 = (30 + 54) % 97
Round 2: 41 = (54 + 84) % 97
Round 3: 28 = (84 + 41) % 97
d1: [24, 30, 54, 84]
d2: [30, 54, 84, 41]
d3: [54, 84, 41, 28]


We append a random padding to te end of the execution trace:

In [194]:
for _ in range(rounds):
    d1.append(random.randint(0, _field_size - 1))
    d2.append(random.randint(0, _field_size - 1))
    d3.append(random.randint(0, _field_size - 1))

d1_trace = np.array(d1)
d2_trace = np.array(d2)
d3_trace = np.array(d3)

print(f"(Padded) Trace Data Columns\n{d1_trace}\n{d2_trace}\n{d3_trace}")

(Padded) Trace Data Columns
[24 30 54 84 17 32 57 48]
[30 54 84 41 72 15 60 26]
[54 84 41 28  8 63 83 12]


## Trace: Control Columns

Initial steps

- Initialization steps are flagged in `c1_trace`
- Computation steps are flagged in `c2_trace`
- Termination step is flagged in `c3_trace`
- 0s at the end of each control column correspond to the padding of the trace

In [195]:
c1 = [1, 0, 0, 0]
c2 = [0, 1, 1, 1]
c3 = [0, 0, 0, 1]

for _ in range(rounds):
    # TODO: Why is it zero and not random?
    c1.append(0)
    c2.append(0)
    c3.append(0)
    # c1.append(random.randint(0, _field_size - 1))
    # c2.append(random.randint(0, _field_size - 1))
    # c3.append(random.randint(0, _field_size - 1))

c1_trace = np.array(c1)
c2_trace = np.array(c2)
c3_trace = np.array(c3)

print("(Padded) Trace Control Columns:\n", c1_trace, "\n", c2_trace, "\n", c3_trace)

(Padded) Trace Control Columns:
 [1 0 0 0 0 0 0 0] 
 [0 1 1 1 0 0 0 0] 
 [0 0 0 1 0 0 0 0]


Now, putting those two together, we will construct a zero-knowledge proof such that:

This trace represents a program that satisfies these 6 rules:
1) Fibonacci words here
2) `d1_trace[0] == 24`  (init 1 constraint)
3) `d2_trace[0] == 30`  (init 2 constraint)
4) `d3_trace[3] == 28`  (termination constraint)
5) `if c2_trace[i] == 1`, then `d2_trace[i] == d1_trace[i+1]`
6) `if c2_trace[i] == 1`, then `d3_trace[i] == d2_trace[i+1]`

In [196]:
trace_data = [d1_trace, d2_trace, d3_trace, c1_trace, c2_trace, c3_trace]
print("Trace Data \n")
for col in trace_data:
    print(col)

Trace Data 

[24 30 54 84 17 32 57 48]
[30 54 84 41 72 15 60 26]
[54 84 41 28  8 63 83 12]
[1 0 0 0 0 0 0 0]
[0 1 1 1 0 0 0 0]
[0 0 0 1 0 0 0 0]


# Lesson 4: Constructing Trace Polynomials

In this lesson, we construct a trace polynomials using Reed-Solomon encoding.

## Trace Polynomials

We run an iNTT on each trace column to construct the associated trace polynomial. The resulting polynomials have at most degree 7 (length of the column minus 1).

In [197]:
d1_coeffs = np.array(intt(d1_trace, prime=_field_size))
d2_coeffs = np.array(intt(d2_trace, prime=_field_size))
d3_coeffs = np.array(intt(d3_trace, prime=_field_size))
c1_coeffs = np.array(intt(c1_trace, prime=_field_size))
c2_coeffs = np.array(intt(c2_trace, prime=_field_size))
c3_coeffs = np.array(intt(c3_trace, prime=_field_size))
print(
    "Coefficients of Trace Polynomials 1, 2, ..., 6:\n",
    d1_coeffs,
    "\n",
    d2_coeffs,
    "\n",
    d3_coeffs,
    "\n",
    c1_coeffs,
    "\n",
    c2_coeffs,
    "\n",
    c3_coeffs,
)

Coefficients of Trace Polynomials 1, 2, ..., 6:
 [19 61 14 14 19 30 17 44] 
 [72 50 62 43 38 50 73 30] 
 [83 56 86 83 12 41 44 37] 
 [85 85 85 85 85 85 85 85] 
 [61 80 12 37 12 60 12 17] 
 [85 89 27 18 12  8 70 79]


# Evaluating Trace Polynomials

Evaluating Trace Polynomials over the powers of 5^12 would return the original trace data:

In [198]:
recovery_domain = EvaluationDomain.from_shift_generator_size(1, 5**12, len(d1_trace))


def recovers_original_data(trace, coeffs):
    return np.all([original == new for (original, new) in zip(trace, recovery_domain.evaluate(coeffs))])


assert recovers_original_data(d1_trace, d1_coeffs)
assert recovers_original_data(d2_trace, d2_coeffs)
assert recovers_original_data(d3_trace, d3_coeffs)
assert recovers_original_data(c1_trace, c1_coeffs)
assert recovers_original_data(c2_trace, c2_coeffs)
assert recovers_original_data(c3_trace, c3_coeffs)

Evaluating Trace Polynomials over the "expanded domain" gives a "trace block":

In [199]:
expanded_domain = EvaluationDomain.from_shift_generator_size(1, 5**3, 32)
d1_rs_expansion = expanded_domain.evaluate(d1_coeffs)
d2_rs_expansion = expanded_domain.evaluate(d2_coeffs)
d3_rs_expansion = expanded_domain.evaluate(d3_coeffs)
c1_rs_expansion = expanded_domain.evaluate(c1_coeffs)
c2_rs_expansion = expanded_domain.evaluate(c2_coeffs)
c3_rs_expansion = expanded_domain.evaluate(c3_coeffs)
print(
    f"Reed Solomon Expansion of Trace Blocks:\n",
    d1_rs_expansion,
    "\n",
    d2_rs_expansion,
    "\n",
    d3_rs_expansion,
    "\n",
    c1_rs_expansion,
    "\n",
    c2_rs_expansion,
    "\n",
    c3_rs_expansion,
)

Reed Solomon Expansion of Trace Blocks:
 [24 69 30 30 30 46 26 33 54 53 22 80 84 16 37 71 17 51 68 64 32 43 36 60
 57 14 76 16 48 54 51 89] 
 [30 49 41  7 54 53 84 37 84  6 44  1 41 29 58 55 72 96 36  6 15 70 61 17
 60 68 19 17 26 11 39 48] 
 [54 74 67 23 84 65 29 23 41 88 40 19 28 54 74  5  8 45 30 45 63 39 67  5
 83 93 87 26 12 12 76 33] 
 [ 1 23 45 53  0 72 83 70  0 10 60 25  0 53 11 68  0  2 62 13  0 26 13 86
  0 46 87 80  0 60 28 91] 
 [ 0 35 31 63  1 32 63 30  1 58 59 20  1  8 91 51  0 38 57 66  0 65 36  9
  0 81 86 70  0 74 65 82] 
 [ 0 26 13 86  0 46 87 80  0 60 28 91  1 23 45 53  0 72 83 70  0 10 60 25
  0 53 11 68  0  2 62 13]


Note that every 4th entry matches the original trace data:

In [200]:
def check_trace_matches(traces, rs_expansions):
    matches = []
    for trace, rs_expansion in zip(traces, rs_expansions):
        for i in range(len(trace)):
            matches.append(trace[i] == rs_expansion[i * 4])
    return matches


is_matches = check_trace_matches(
    [d1_trace, d2_trace, d3_trace, c1_trace, c2_trace, c3_trace],
    [d1_rs_expansion, d2_rs_expansion, d3_rs_expansion, c1_rs_expansion, c2_rs_expansion, c3_rs_expansion],
)
assert np.all(is_matches)

This is a degree 4 Reed-Solomon expansion of the original trace:

In [201]:
trace_rs_expansion = np.array(
    [
        d1_rs_expansion,
        d2_rs_expansion,
        d3_rs_expansion,
        c1_rs_expansion,
        c2_rs_expansion,
        c3_rs_expansion,
    ]
)

# Lesson 5: ZK Commitments of the Trace Data

To maintain a zero-knowledge protocol, the trace polynomials are evaluated over a "zk commitment domain", {5^1, 5^4, ..., 5^94}.

Evaluating Trace Polynomials over the zk commitment domain gives "zero-knowledge trace blocks."

In [202]:
zk_commitment_domain = EvaluationDomain.from_shift_generator_size(5, 5**3, 32)
d1_zk_commitment = zk_commitment_domain.evaluate(d1_coeffs)
d2_zk_commitment = zk_commitment_domain.evaluate(d2_coeffs)
d3_zk_commitment = zk_commitment_domain.evaluate(d3_coeffs)
c1_zk_commitment = zk_commitment_domain.evaluate(c1_coeffs)
c2_zk_commitment = zk_commitment_domain.evaluate(c2_coeffs)
c3_zk_commitment = zk_commitment_domain.evaluate(c3_coeffs)
print(
    "Zero-Knowledge Trace Blocks:\n",
    d1_zk_commitment,
    "\n",
    d2_zk_commitment,
    "\n",
    d3_zk_commitment,
    "\n",
    c1_zk_commitment,
    "\n",
    c2_zk_commitment,
    "\n",
    c3_zk_commitment,
)

Zero-Knowledge Trace Blocks:
 [44 12 60 87 70 36  7 14 57 49 30 48 11 73 25 71 78 79 42 17 87 36  1 10
 61 53 60 73 35  8 24 26] 
 [75 28 40 65 21 94 14 67 19 83 69 38  7 64 58 33 39 18 67 88 74 54 70  2
 95  5 53  7 52 36 11 82] 
 [92 34 18  3  4 73 42 75 39 56 36 74 12 40 42 48 96 84 57 47 60  2 20 29
 35 43 39  6 35 41 22 91] 
 [82 18 32 68 81 41 18 86 92 59 14 44 43 31  8 92 10 71 50 89  2 32 65 22
 32 16 38 50 47 24 67 35] 
 [81 72 73 10 64 58 40 56 16 83 20 92 61 21 64  4 22 34 40 28 48 64 72 31
 55 37 26  9 44 22 56 64] 
 [ 2 32 65 22 32 16 38 50 47 24 67 35 82 18 32 68 81 41 18 86 92 59 14 44
 43 31  8 92 10 71 50 89]


These zk-commitment blocks do not share any evaluation points with the original trace data:


In [203]:
is_matches = check_trace_matches(
    [d1_trace, d2_trace, d3_trace, c1_trace, c2_trace, c3_trace],
    [d1_zk_commitment, d2_zk_commitment, d3_zk_commitment, c1_zk_commitment, c2_zk_commitment, c3_zk_commitment],
)
assert not np.any(is_matches)

These evaluations are then committed to Merkle Trees (one for the data blocks and one for the control blocks). The term "polynomial commitment" describes this process of a Merkle commitment where the leaves of the tree are evaluations of a polynomial. The root of these two Merkle Trees are the first entries on the RISC Zero seal. The third entry on the RISC Zero seal is an `Accum` commitment; that commitment is not necessary in this simplified example.

In [204]:
trace_zk_commitment = np.array(
    [
        d1_zk_commitment,
        d2_zk_commitment,
        d3_zk_commitment,
        c1_zk_commitment,
        c2_zk_commitment,
        c3_zk_commitment,
    ]
)

# Lesson 6: Constraint Polynomials

The constraints are used to check the correctness of the trace. In this example, we check 6 rules to establish the validity of the trace.

Applying rule checks to trace blocks creates constraint blocks. A constraint block has 0s in every 4th row - These 0s indicate the passing of the various rule-checks.

Applying rule checks to zk-commitment trace blocks makes zk-commitment constraint blocks. Similarly, applying rule checks to trace polynomials makes constraint polynomials. In code, this happens in terms of trace blocks.

In [205]:
def fib_constraint(trace):
    # trace[2] - trace[1] - trace[0] returns 0 when the fibonacci addition relation holds.
    # trace[3] + trace[4] + trace[5] returns 0 when we aren't enforcing this rule.
    # The product of these two terms returns 0 at each row of the original trace.
    constraint = np.multiply((trace[2] - trace[1] - trace[0]), trace[3] + trace[4] + trace[5]) % _field_size
    return constraint


fib_constraint_columns = fib_constraint(trace_data)
fib_constraint_rs_expansion = fib_constraint(trace_rs_expansion)
fib_constraint_zk_commitment = fib_constraint(trace_zk_commitment)

print(
    "Applied to the original trace data, the constraint yields all 0s: \n",
    fib_constraint_columns,
)
print(
    "Applied to the Reed-Solomon expanded trace blocks, the constraint yields 0s in every 4th row: \n",
    fib_constraint_rs_expansion,
)

Applied to the original trace data, the constraint yields all 0s: 
 [0 0 0 0 0 0 0 0]
Applied to the Reed-Solomon expanded trace blocks, the constraint yields 0s in every 4th row: 
 [ 0 87 32 82  0 41 42 76  0 26 58  7  0 77 17 43  0 22 87 58  0 92 28 90
  0 40 80 26  0 67 61 56]


In practice, we apply these rule-checks to the trace blocks (i.e., the evaluations of the trace polynomials over the zk-commitment domain).

Although the zeros are no longer apparent, this block still corresponds to the evaluation of a polynomial whose degree is bounded by $2l$ where $l$ is the length of the trace.

The 6 constraint blocks and the associated polynomials, along with the coefficients of the associated polynomials, are shown below:

In [206]:
# Constraint the first Fibonacci input
def init1_constraint(trace):
    # trace[0] - 24 returns 0 when the value loaded into the program matches the asserted input of 24
    # trace[3] returns 0 when we aren't enforcing this rule.
    # The product of these two terms returns 0 at each row of the original trace.
    constraint = np.multiply(trace[0] - 24, trace[3]) % _field_size
    return constraint


init1_constraint_columns = init1_constraint(trace_data)
init1_constraint_rs_expansion = init1_constraint(trace_rs_expansion)
init1_constraint_zk_commitment = init1_constraint(trace_zk_commitment)


# Constraint the second Fibonacci input
def init2_constraint(trace):
    # trace[1] - 30 returns 0 when the value loaded into the program matches the asserted input of 30
    # trace[3] returns 0 when we aren't enforcing this rule.
    # The product of these two terms returns 0 at each row of the original trace.
    constraint = np.multiply(trace[1] - 30, trace[3]) % _field_size
    return constraint


init2_constraint_columns = init2_constraint(trace_data)
init2_constraint_rs_expansion = init2_constraint(trace_rs_expansion)
init2_constraint_zk_commitment = init2_constraint(trace_zk_commitment)


# Constraint the Fibonacci output
def termination_constraint(trace):
    # trace[2] - 28 returns 0 when the value loaded into the program matches the asserted input of 30
    # trace[5] returns 0 when we aren't enforcing this rule.
    # The product of these two terms returns 0 at each row of the original trace.
    constraint = np.multiply(trace[2] - 28, trace[5]) % _field_size
    return constraint


termination_constraint_columns = termination_constraint(trace_data)
termination_constraint_rs_expansion = termination_constraint(trace_rs_expansion)
termination_constraint_zk_commitment = termination_constraint(trace_zk_commitment)


# Constraint the Fibonacci intermediate compute step for the first previous input
def first_recursion_constraint(trace):
    # We check that the ith term in trace[0] matches the "previous" term in trace[1].
    # We use "previous" to mean "one computational step before."
    # i.e., for trace_columns, we're checking trace[0][i] against trace[0][i-1], while
    # for trace_zk_commitment and trace_rs_expansion, we're checking trace[0][i] vs. trace[1][i-4]
    # We express this length-dependent step-size as len(trace[0])//8
    # We also use "previous" in a cyclical sense, wrapping around to the end of the trace.
    # Putting this together, the relation we are checking is:
    #   trace[0][i] - trace[1][i - len(trace[0])//8 % len(trace[0])
    # trace[4] is the associated indicator column; trace[4][i] = 0 when this rule isn't enforced.
    constraint = np.zeros(len(trace[0]), int)
    for i in range(len(trace[0])):
        constraint[i] = ((trace[0][i] - trace[1][i - len(trace[0]) // 8 % len(trace[0])]) * trace[4][i]) % _field_size
        constraint = np.array(constraint)
    return constraint


first_recursion_constraint_columns = first_recursion_constraint(trace_data)
first_recursion_constraint_rs_expansion = first_recursion_constraint(trace_rs_expansion)
first_recursion_constraint_zk_commitment = first_recursion_constraint(trace_zk_commitment)


# Constraint the Fibonacci intermediate compute step for the second previous input
def second_recursion_constraint(trace):
    # We check that the ith term in trace[1] matches the "previous" term in trace[2].
    # See first_recursion_constraint for explanation of "previous."
    # trace[4] is the associated indicator column; trace[4][i] = 0 when this rule isn't enforced.
    constraint = np.zeros(len(trace[0]), int)
    for i in range(len(trace[0])):
        constraint[i] = ((trace[1][i] - trace[2][i - len(trace[0]) // 8 % len(trace[0])]) * trace[4][i]) % _field_size
        constraint = np.array(constraint)
    return constraint


second_recursion_constraint_columns = second_recursion_constraint(trace_data)
second_recursion_constraint_rs_expansion = second_recursion_constraint(trace_rs_expansion)
second_recursion_constraint_zk_commitment = second_recursion_constraint(trace_zk_commitment)

print("Constraint Blocks:")
print("Fib Constraint: \n", fib_constraint_zk_commitment)
print("Init1 Constraint: \n", init1_constraint_zk_commitment)
print("Init2 Constraint: \n", init2_constraint_zk_commitment)
print("Termination Constraint: \n", termination_constraint_zk_commitment)
print("First Recursion Constraint: \n", first_recursion_constraint_zk_commitment)
print(
    "Second Recursion Constraint: \n",
    second_recursion_constraint_zk_commitment,
)

Constraint Blocks:
Fib Constraint: 
 [ 7 44 28 38 24 41 76 12 85 91 39 82 48  0  4 31 52 42 10 60 14 37 59  0
 81  1  7 78 83 37 79  5]
Init1 Constraint: 
 [88 75 85 16 40  7 82 13 29 20 84 86 23 64  8 56 55 25 27 56 29 93 57 80
 20 76 10 25 32  4  0 70]
Init2 Constraint: 
 [ 4 61 29 52 47  5  3 78 55 23 61 61 78 84 30 82 90 21  7 21 88 89 78 63
 43 85  1 14 64 47 85 74]
Termination Constraint: 
 [31 95 29 32  8 41 47 22 32 90 51 58 46 22 60  2 76 65 37 82 34 18 82 44
 10 77 88 13 70 50 88 78]
First Recursion Constraint: 
 [31 18 85 50 68 76 38 54 91 48 29 95 94 81 94 35 10 25 39 37 73 85  1  7
 61 60 31 57 76 66 25 52]
Second Recursion Constraint: 
 [39 34 53 31 15 85 34 92 46 54 55 88 85 71 50 30 12 28 30 53 11 20 63 60
 82 14 82 93 69 40 81 14]


# Lesson 7: Mixing Constraint Polynomials

We mix the constraint polynomials into a single mixed constraint polynomial. In practice, this is a mix of evaluations over the zk-commitment domain. For clarity, we show the Mixed Constraint Polynomial as evaluations over the same three domains as above.

In [207]:
mixing_parameter = 3  # Normally, we would calculate this from a Merkle Tree root

mixed_constraint_poly_columns = mix(
    [
        fib_constraint_columns,
        init1_constraint_columns,
        init2_constraint_columns,
        termination_constraint_columns,
        first_recursion_constraint_columns,
        second_recursion_constraint_columns,
    ],
    mixing_parameter,
)
mixed_constraint_poly_rs_expansion = mix(
    [
        fib_constraint_rs_expansion,
        init1_constraint_rs_expansion,
        init2_constraint_rs_expansion,
        termination_constraint_rs_expansion,
        first_recursion_constraint_rs_expansion,
        second_recursion_constraint_rs_expansion,
    ],
    mixing_parameter,
)
mixed_constraint_poly_zk_commitment = mix(
    [
        fib_constraint_zk_commitment,
        init1_constraint_zk_commitment,
        init2_constraint_zk_commitment,
        termination_constraint_zk_commitment,
        first_recursion_constraint_zk_commitment,
        second_recursion_constraint_zk_commitment,
    ],
    mixing_parameter,
)

Although they stay in evaluation form throughout the protocol, we show the constraint polynomials in coefficient form for didactic clarity and troubleshooting. To convert between coefficient form and evaluation form of a given polynomial, we use iNTTs and NTTs.

Computing coefficients of Constraint Polynomials:

In [208]:
fib_constraint_poly_coeffs = intt(fib_constraint_rs_expansion, prime=_field_size)
init1_constraint_poly_coeffs = intt(init1_constraint_rs_expansion, prime=_field_size)
init2_constraint_poly_coeffs = intt(init2_constraint_rs_expansion, prime=_field_size)
termination_constraint_poly_coeffs = intt(termination_constraint_rs_expansion, prime=_field_size)
first_recursion_constraint_poly_coeffs = intt(first_recursion_constraint_rs_expansion, prime=_field_size)
second_recursion_constraint_poly_coeffs = intt(second_recursion_constraint_rs_expansion, prime=_field_size)
mixed_constraint_poly_coeffs = intt(mixed_constraint_poly_rs_expansion, prime=_field_size)

print("Coefficients of Constraint Polynomials:")
print("Fibonacci Constraint: ", fib_constraint_poly_coeffs)
print("Init1 Constraint: ", init1_constraint_poly_coeffs)
print("Init2 Constraint: ", init2_constraint_poly_coeffs)
print("Termination Constraint: ", termination_constraint_poly_coeffs)
print("First Recursion Constraint: ", first_recursion_constraint_poly_coeffs)
print("Second Recursion Constraint: ", second_recursion_constraint_poly_coeffs, "\n")
print("Mixed Constraint Coefficients: \n", mixed_constraint_poly_coeffs, "\n")

Coefficients of Constraint Polynomials:
Fibonacci Constraint:  [92, 7, 55, 24, 32, 52, 4, 0, 5, 90, 42, 73, 65, 45, 93, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Init1 Constraint:  [60, 7, 33, 59, 25, 53, 43, 0, 37, 90, 64, 38, 72, 44, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Init2 Constraint:  [78, 60, 92, 61, 90, 72, 69, 0, 19, 37, 5, 36, 7, 25, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Termination Constraint:  [19, 52, 5, 42, 78, 45, 84, 0, 78, 45, 92, 55, 19, 52, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
First Recursion Constraint:  [65, 79, 18, 53, 85, 55, 76, 0, 32, 18, 79, 44, 12, 42, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Second Recursion Constraint:  [8, 19, 93, 49, 41, 9, 73, 0, 89, 78, 4, 48, 56, 88, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

Mixed Constraint Coefficients: 
 [63, 87, 51, 42, 83, 83, 48, 0, 34, 10, 46, 55, 14, 14, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 



We now introduce the Zeros Polynomial, which is used to compute the Validity Polynomial.

The Zeros Polynomial is $Z(x) = x^8 - 1$

Remember the "difference of perfect squares" from Algebra II?

It turns out that this "difference of perfect 8th powers" works out very nicely over finite fields.

$Z(x) = (x - 5^0)(x - 5^{12})(x - 5^{24})...(x-5^{84})$

The next term in the pattern would be $(x-5^{96})$, which "wraps around" quite nicely since $5^0 = 5^{96}$.

In [209]:
zeros_poly_coeffs = [-1, 0, 0, 0, 0, 0, 0, 0, 1]
zeros_poly_evals_rs_expansion = expanded_domain.evaluate(zeros_poly_coeffs)
zeros_poly_evals_zk_commitment = zk_commitment_domain.evaluate(zeros_poly_coeffs)
print("Zeros Polynomial coefficients: \n", zeros_poly_coeffs)
print(
    "Zeros Polynomial evaluated on Reed Solomon expansion domain: \n",
    zeros_poly_evals_rs_expansion,
)
print(
    "Zeros Polynomial evaluated on zk-commitment domain: \n",
    zeros_poly_evals_zk_commitment,
    "\n",
)

# Constructing reciprocals of Zeros Polynomial evals to facilitate computation
reciprocals_of_zeros_poly = reciprocal_elementwise(zeros_poly_evals_zk_commitment)

Zeros Polynomial coefficients: 
 [-1, 0, 0, 0, 0, 0, 0, 0, 1]
Zeros Polynomial evaluated on Reed Solomon expansion domain: 
 [ 0 21 95 74  0 21 95 74  0 21 95 74  0 21 95 74  0 21 95 74  0 21 95 74
  0 21 95 74  0 21 95 74]
Zeros Polynomial evaluated on zk-commitment domain: 
 [ 5 34 90 61  5 34 90 61  5 34 90 61  5 34 90 61  5 34 90 61  5 34 90 61
  5 34 90 61  5 34 90 61] 



# Lesson 8: The core of the RISC Zero STARK

## Constructing Validity Polynomial

The Validity Polynomial is the quotient of the Mixed Constraint Polynomial and the Zeros Polynomial. 

The Prover constructs the Validity Polynomial by dividing the Mixed Constraint Polynomial by the publicly known Zeros Polynomial: $V(x) = C(x)/Z(x)$

These evaluations are committed to a Merkle tree, and the root of that tree is appended to the seal.
The rest of the protocol aims to prove that the commitments for the trace polynomials and the validity polynomials are actually evaluations of low-degree polynomials that satisfy V=C/Z as asserted."

Constructing coefficients of validity polynomial for use in DEEP:

In [210]:
validity_polynomial_evals = np.multiply(mixed_constraint_poly_zk_commitment, reciprocals_of_zeros_poly) % _field_size
print(
    "Evaluations of Validity Polynomial on the zk-commitment domain \n",
    validity_polynomial_evals,
    "\n",
)

evalmatrix_inv = np.linalg.inv(_finite_field_array(zk_commitment_domain.as_vandermonde_matrix()))
validity_polynomial_coeffs = np.array(evalmatrix_inv @ _finite_field_array(validity_polynomial_evals))

Evaluations of Validity Polynomial on the zk-commitment domain 
 [85 63 91  8 86 34 75 85 39  6  2 24 52  4 62 90  3 35 47 76 23 16 68 25
 89 11 10 51 89  6 14 10] 



# Lesson 9: The DEEP Technique

The DEEP technique is a means of improving the security level associated with a single query by sampling from a larger domain than the commitment domain.

The Prover provides information to convince the verifier that the Merkle commitments satisfy the relation $V(x)=C(x)/Z(x)$, where $x$ is at the `DEEP_test_point`.

In [211]:
DEEP_test_point = 93  # TODO: How to select this point?

# The DEEP Polynomials all share the following denominator:
DEEP_denom = (zk_commitment_domain.domain - DEEP_test_point) % _field_size

# We rewrite DEEP_denom in terms of reciprocals to facilitate computation later:
DEEP_denom = reciprocal_elementwise(DEEP_denom)

# TODO: Implement Interpolation of taps for DEEP_poly2 & DEEP_poly3

DEEP_poly1_evals = DEEPquotient(
    d1_zk_commitment,
    zk_commitment_domain.domain,
    DEEP_test_point,
    evaluate_at_point(d1_coeffs, DEEP_test_point),
)
# DEEP_poly2 = hard-coded for now
# DEEP Poly 3 = hard-coded for now
DEEP_poly4_evals = DEEPquotient(
    c1_zk_commitment,
    zk_commitment_domain.domain,
    DEEP_test_point,
    evaluate_at_point(c1_coeffs, DEEP_test_point),
)
DEEP_poly5_evals = DEEPquotient(
    c2_zk_commitment,
    zk_commitment_domain.domain,
    DEEP_test_point,
    evaluate_at_point(c2_coeffs, DEEP_test_point),
)
DEEP_poly6_evals = DEEPquotient(
    c3_zk_commitment,
    zk_commitment_domain.domain,
    DEEP_test_point,
    evaluate_at_point(c3_coeffs, DEEP_test_point),
)
DEEPValidity_evals = DEEPquotient(
    validity_polynomial_evals,
    zk_commitment_domain.domain,
    DEEP_test_point,
    evaluate_at_point(validity_polynomial_coeffs, DEEP_test_point),
)

# Hard-coding DEEP Polynomial evaluations for now
DEEP_poly2_evals = [
    19,
    74,
    82,
    13,
    48,
    30,
    17,
    58,
    93,
    63,
    64,
    42,
    45,
    10,
    21,
    71,
    1,
    52,
    72,
    71,
    60,
    10,
    8,
    44,
    41,
    5,
    14,
    16,
    49,
    15,
    78,
    41,
]

# Hard-coding DEEP Polynomial evaluations for now
DEEP_poly3_evals = [
    84,
    63,
    89,
    93,
    75,
    36,
    11,
    80,
    44,
    23,
    40,
    11,
    80,
    33,
    38,
    50,
    72,
    33,
    35,
    78,
    8,
    8,
    86,
    36,
    9,
    25,
    88,
    95,
    65,
    22,
    50,
    91,
]

print("Interpolation of DEEP Poly 1:\n", intt(DEEP_poly1_evals, _field_size))
print("Interpolations of DEEP_poly2: \n", intt(DEEP_poly2_evals, _field_size))
print("Interpolations of DEEP_poly3: \n", intt(DEEP_poly3_evals, _field_size))
print("interpolation of DEEP poly 4: \n", intt(DEEP_poly4_evals, _field_size))
print("interpolation of DEEP poly 5:\n", intt(DEEP_poly5_evals, _field_size))
print("interpolation of DEEP poly 6:\n", intt(DEEP_poly6_evals, _field_size))
print(
    "interpolation of DEEP Validity Polynomial:\n",
    intt(DEEPValidity_evals, _field_size),
)

Interpolation of DEEP Poly 1:
 [50, 38, 40, 48, 23, 56, 61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Interpolations of DEEP_poly2: 
 [93, 39, 20, 48, 91, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Interpolations of DEEP_poly3: 
 [91, 94, 50, 50, 34, 56, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
interpolation of DEEP poly 4: 
 [58, 58, 95, 64, 82, 77, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
interpolation of DEEP poly 5:
 [4, 95, 29, 53, 87, 85, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
interpolation of DEEP poly 6:
 [89, 0, 96, 6, 73, 72, 50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
interpolation of DEEP Validity Polynomial:
 [69, 96, 22, 18, 31, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Lesson 10: Mixing (Batching) for FRI

The initial FRI polynomial is the mix of the 7 DEEP polynomials.

In [212]:
fri_input = mix(
    [
        DEEP_poly1_evals,
        DEEP_poly2_evals,
        DEEP_poly3_evals,
        DEEP_poly4_evals,
        DEEP_poly5_evals,
        DEEP_poly6_evals,
        DEEPValidity_evals,
    ],
    21,
)

Interpolation of initial FRI Input

In [213]:
print(intt(fri_input, _field_size))

[81, 94, 68, 81, 82, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


With the "batching" complete, we're ready for the FRI commit rounds.

# Lesson 11: FRI Commit Rounds

In this part of the protocol, the Prover provides information to convince the verifier that the DEEP polynomials are low-degree.

It follows that the Trace & Validity polynomials are also low-degree.

In [214]:
mixing_parameters_fri = np.array([1, 3, 22])
fri_rounds = len(mixing_parameters_fri)

# Run (fri_rounds) rounds of FRI commit
print("FRI Commit Phase")
final_fri_block = EvaluationDomain(
    fri(
        fri_input,
        zk_commitment_domain,
        mixing_parameters_fri,
        fri_rounds,
        fri_rounds,
    )
).evaluation_domain_of_half_size.domain

FRI Commit Phase

--- begin round 1 ---
D_super_( 0 ): [ 5 43 40 53 29 36 38 94 13 73  7  2 56 16 60 31 92 54 57 44 68 61 59  3
 84 24 90 95 41 81 37 66]
Split blocks: 
 [15 36 34 22 43 66 25 86 10 88 24 80 62 37 47 39] 
 [88 68  5 83 28 73 45 15 66 86 52 71 29 81 12 42]
Now mix those blocks using the mix parameter 1
FRI Commitment (aka f_super_( 1 ): [ 6  7 39  8 71 42 70  4 76 77 76 54 91 21 59 81]

--- begin round 2 ---
D_super_( 1 ): [np.int64(5), np.int64(40), np.int64(29), np.int64(38), np.int64(13), np.int64(7), np.int64(56), np.int64(60), np.int64(92), np.int64(57), np.int64(68), np.int64(59), np.int64(84), np.int64(90), np.int64(41), np.int64(37)]
Split blocks: 
 [57 96 70 55 65 26 52 67] 
 [91 51 13  6 43 83 24 31]
Now mix those blocks using the mix parameter 3
FRI Commitment (aka f_super_( 2 ): [39 55 12 73  0 81 27 63]

--- begin round 3 ---
D_super_( 2 ): [np.int64(5), np.int64(29), np.int64(13), np.int64(56), np.int64(92), np.int64(68), np.int64(84), np.int64(41)]
Split b

The Prover sends this final FRI Commitment in full, rather than as a hash.

The Verifier can confirm manually that the final FRI commitment is indeed a low-degree polynomial.

# Lesson 12: FRI Query Phase

The FRI Query phase confirms the correctness of the folding at each round of the FRI commit phase.
In the Batched FRI protocol, the query phase also checks the correctness of the FRI batching.

We only show a single query, and we only instantiate the check for the final round of FRI folding.
Future improvements may extend this to include a more complete implementation of FRI queries.

The notation here matches what's described on page 39 of Proximity Gaps for Reed-Solomon Codes.
For a more complete description of the query phase, see https://eprint.iacr.org/2020/654

## Specifying query location

The verifier specifies a query location `g_super_(3)` from `D_super_(3)`


In [215]:
g_super_3_index = 25
g_super_3 = mod_pow(5, g_super_3_index)
print("Choose g_super_(3) = 5^{} = {}.\n".format(g_super_3_index, g_super_3))

Choose g_super_(3) = 5^25 = 13.



Now, the prover provides 2 evaluations from `f_super_(2)`. The verifier will use these evaluations to confirm the purported value of `f_super_(3)`. The details here are rather hairy:

## Query location determines coset

The choice of `g_super_3` determines a coset `C_super_2`.


In [216]:
print(
    "C_super_(2) = {{ g : g/5 is a square root of (g_super_3)/5 }} = {{ 5^{}, 5^{} }}".format(
        (g_super_3_index - 1) // 2 + 1,
        (g_super_3_index - 1) // 2 + (_field_size + 1) // 2,
    )
)
C_super_2 = EvaluationDomain(
    [
        mod_pow(5, ((g_super_3_index - 1) // 2 + 1)),
        mod_pow(5, ((g_super_3_index - 1) // 2 + (_field_size + 1) // 2)),
    ]
)

C_super_(2) = { g : g/5 is a square root of (g_super_3)/5 } = { 5^13, 5^61 }


The prover provides the evaluations of `f_super_2` on each of the elements of `C_super_2`.

The coset `C_super_2` determines a Vandermonde Matrix `M_super_2`.

In [217]:
M_super_2 = np.linalg.inv(_finite_field_array(C_super_2.as_vandermonde_matrix()))
print("M_super_2 = inverse_vandermonde(C_super_2) = \n", M_super_2)

# boldface_z_super_2 is the vector of powers of the mixing parameters
boldface_z_super_2 = EvaluationDomain([1, mixing_parameters_fri[2]])

# Checking folding
# See page 39 of Proximity Gaps for RS Codes
expected = np.dot(
    _finite_field_array(boldface_z_super_2.domain),
    (M_super_2 @ _finite_field_array(final_fri_block)),
)

# Reject if f_super_3 evaluated @ g_super_(3) != expected
# Evaluate the polynomial at g_super_3 in the finite field
f_super_3_at_g_super_3 = np.polyval(final_fri_block, g_super_3) % _field_size
print(expected, "==", f_super_3_at_g_super_3)
# assert expected == f_super_3_at_g_super_3 # TODO: Fix

M_super_2 = inverse_vandermonde(C_super_2) = 
 [[49 49]
 [92  5]]
18 == 58


Now, the verifier can use the check shown in step (3) of `A single invocation of the batched QUERY phase.`

Check the source code or page 39 of the ProxGaps paper to see a detailed description of this check.

The verifier computes an `expected` value for f₃ evaluated at g₃ and checks the result against the purported value provided earlier.

This check confirms the correctness of the folding from f₂ to f₃.

A more complete demonstration of a FRI query would include an analogous check to confirm the correctness of each round of FRI folding, and a check to confirm the initial FRI batching.

---

**Congratulations!**  
You've just run a (mostly) complete STARK protocol, including DEEP-ALI, FRI committing, and a FRI query.

For a more visual & more verbose walk-through of this example, check out our STARK by Hand explainer:  
[https://dev.risczero.com/proof-system/stark-by-hand](https://dev.risczero.com/proof-system/stark-by-hand)

We also have videos at [https://dev.risczero.com/studyclub](https://dev.risczero.com/studyclub) that walk through a large portion of these ideas.

We hope you find these resources useful - Happy learning!  
Got questions/feedback? Find us on Discord: [https://discord.gg/risczero](https://discord.gg/risczero)